# **Winelists**

In [ ]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://nairobidrinks.co.ke/pricelist'
r = requests.get(url)

<Response [200]>

In [ ]:
soup = BeautifulSoup(r.text, 'lxml')
sects = soup.find_all('tr')

In [ ]:
titles = []
for tr in sects:
    title = tr.find_all('td')
    titles.extend(title)

In [ ]:
products = [title.text.strip() for title in titles]
caps = products[0::3]
prices = products[1::3]

In [ ]:
prod_split = [cap.split('\n') for cap in caps]
names = [split[0] for split in prod_split]
vol = [split[1] for split in prod_split]

full_list = {
    'Name': names,
    'Volume': vol,
    'Price': prices
}

In [ ]:
df = pd.DataFrame(full_list)
df.drop_duplicates()
df.to_csv('Liqourlist.csv', index=False)

In [ ]:
viewdf = pd.read_csv('Liqourlist.csv')
viewdf

,Name,Volume,Price
0,Jack Daniels Tennessee Cola,330 ml,Ksh 460
1,Best whisky,750 ml,"Ksh 1,400"
2,Black & white whisky,750 ml,"Ksh 1,499"
3,William lawson,750 ml,"Ksh 1,800"
4,Sir Edwards,750 ml,"Ksh 1,800"
...,...,...,...
2581,Dom perignon brut luminous,750 ml,"Ksh 59,999"
2582,Dom perignon brut 20.. vintage,750 ml,"Ksh 59,999"
2583,Dom perignon rose,750 ml,"Ksh 64,999"
2584,Armand de brignac ace of spades brut gold cham...,750 ml,"Ksh 64,999"


# **Wholesalers**

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'https://jayswines.com/categories/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

urlists = soup.find_all('a', {'class':'vc_column-link smooth-scroll'})
links = [url.get('href') for url in urlists]
links = links[:-2]

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')

    # Extract product names and prices
    wraps = soup.find_all('div', {'class':'w-vwrapper usg_vwrapper_1 align_center valign_top'})
    names = [wrap.find('a') for wrap in wraps]
    name = [name.text.strip() for name in names]

    price_spans = [wrap.find('span', {'class':'woocommerce-Price-amount amount'}) for wrap in wraps]
    price = [price_span.text.strip() for price_span in price_spans]

    data = {
        'Name': name,
        'Price': price
    }

    df = pd.DataFrame(data)
    df.drop_duplicates()

    # Get the name from the link
    product_name = link.split('/')[-2]
    product_name = product_name.upper()

    # Save the data to a CSV file
    df.to_csv(f'{product_name}.csv', index=False)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize an empty list to store DataFrames
all_dfs = []

url = 'https://jayswines.com/categories/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

urlists = soup.find_all('a', {'class': 'vc_column-link smooth-scroll'})
links = [url.get('href') for url in urlists]
links = links[:-2]

for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')

    # Extract product names and prices
    wraps = soup.find_all('div', {'class': 'w-vwrapper usg_vwrapper_1 align_center valign_top'})
    names = [wrap.find('a') for wrap in wraps]
    name = [name.text.strip() for name in names]

    price_spans = [wrap.find('span', {'class': 'woocommerce-Price-amount amount'}) for wrap in wraps]
    price = [price_span.text.strip() for price_span in price_spans]

    # Extract the liquor type from the link
    liquor_type = link.split('/')[-2].replace('-', ' ').title()

    data = {
        'Name': name,
        'Price': price,
        'Liquor Type': [liquor_type] * len(name)  # Add the liquor type column
    }

    df = pd.DataFrame(data)
    df.drop_duplicates(inplace=True)

    # Append the DataFrame to the list
    all_dfs.append(df)

    # Save each DataFrame to a CSV file (optional)
    df.to_csv(f'{liquor_type}.csv', index=False)

# Concatenate all DataFrames into one
final_df = pd.concat(all_dfs, ignore_index=True)

# Save the final DataFrame to a CSV file
final_df.to_csv('all_liquors.csv', index=False)

# Print the final DataFrame
print(final_df)


                                                   Name        Price  \
0     SIMONSIG FRANS MALAN (COMPLEX CAPE STYLE BLEND...  KSh6,750.00   
1         NAPA CELLARS CABERNET SAUVIGNON 750ML (14.2%)  KSh6,400.00   
2                    NAPA CELLARS CHARDONNY 750ML (14%)  KSh6,400.00   
3                   SIMONSIG TIARA DRY RED 750ML(14.5%)  KSh6,400.00   
4            GRAHAMS FINE 10YEARS TAWNY PORT 750ML(20%)  KSh6,200.00   
...                                                 ...          ...   
1047                                     V&A 750ML(20%)    KSh950.00   
1048                          JELZIN APPLE 750ML(16.6%)  KSh1,100.00   
1049                          GRENADINE SYRUP 750ML(0%)    KSh450.00   
1050                              BEST CREAM 250ML(17%)    KSh430.00   
1051                                     V&A 250ML(20%)    KSh370.00   

     Liquor Type  
0           Wine  
1           Wine  
2           Wine  
3           Wine  
4           Wine  
...          ...  
10

In [9]:
final_df[400:]

,Name,Price,Liquor Type
400,ROBERTSON SWEET RED 750 ML(8%),"KSh1,350.00",Wine
401,FRONTERA CHARDONNAY 750ML(12%),"KSh1,350.00",Wine
402,MARA ROSE 750ML(8%),"KSh1,350.00",Wine
403,CINZANO ROSSO 750ML,"KSh1,350.00",Wine
404,GATO NEGRO SWEET MOSCATO 750ML(12%),"KSh1,350.00",Wine
...,...,...,...
1047,V&A 750ML(20%),KSh950.00,Liqueur
1048,JELZIN APPLE 750ML(16.6%),"KSh1,100.00",Liqueur
1049,GRENADINE SYRUP 750ML(0%),KSh450.00,Liqueur
1050,BEST CREAM 250ML(17%),KSh430.00,Liqueur
